These finetuning steps are based on the tutorial linked here: https://medium.com/nlplanet/bert-finetuning-with-hugging-face-and-training-visualizations-with-tensorboard-46368a57fc97 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import scraper 
from sklearn.preprocessing import LabelEncoder

In [2]:
#getting data ready, selecting classes (most common answers)
df = scraper.hf_dataframe()
qa = df[["question", "answer"]]
qa_clean = qa.applymap(scraper.cleaner) 

top_answers = qa_clean['answer'].value_counts().head(20).index.tolist()

df_filtered = qa_clean[qa_clean['answer'].isin(top_answers)].copy()

le = LabelEncoder()
df_filtered['label'] = le.fit_transform(df_filtered['answer'])

print("Most commonly occuring answers: ", le.classes_)

C:\Users\sjkov\AppData\Local\Temp\ipykernel_13852\870503883.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  qa_clean = qa.applymap(scraper.cleaner)


Most commonly occuring answers:  ['alaska' 'australia' 'california' 'canada' 'chicago' 'china' 'florida'
 'france' 'germany' 'hawaii' 'india' 'ireland' 'italy' 'japan' 'mexico'
 'paris' 'russia' 'south africa' 'spain' 'texas']


In [3]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df_filtered[['question', 'label']])

dataset = hf_dataset.train_test_split(test_size=0.2)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'label', '__index_level_0__'],
        num_rows: 2700
    })
    test: Dataset({
        features: ['question', 'label', '__index_level_0__'],
        num_rows: 675
    })
})


In [4]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

#preprocessing

def preprocess_function(examples):
    return tokenizer(examples["question"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

Map:   0%|          | 0/675 [00:00<?, ? examples/s]

In [5]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

to_label = {}
to_id = {}

for index, label_name in enumerate(le.classes_):
    to_label[index] = label_name
    to_id[label_name] = index

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=20,
    id2label=to_label,
    label2id=to_id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#figure out tensorboard setup here

In [8]:
from transformers import TrainingArguments, Trainer
#bert finetuning here
#experiment with different parameters
#definitely need to mess around with these
args = TrainingArguments(
    output_dir="bert_jeopardy_model",
    eval_strategy="steps",    
    eval_steps=50,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=200,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="tensorboard"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("training")
trainer.train()

C:\Users\sjkov\AppData\Local\Temp\ipykernel_13852\1081196706.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


training


C:\Users\sjkov\miniforge3\envs\ml\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Accuracy
50,2.984400,2.955688,0.056296
100,2.842800,2.711948,0.336296
150,2.620800,2.553643,0.431111


C:\Users\sjkov\miniforge3\envs\ml\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\sjkov\miniforge3\envs\ml\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=169, training_loss=2.7841867300180287, metrics={'train_runtime': 759.1251, 'train_samples_per_second': 3.557, 'train_steps_per_second': 0.223, 'total_flos': 89444196864000.0, 'train_loss': 2.7841867300180287, 'epoch': 1.0})